In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.common import ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.common.by import By
import time


#retrieve code of webpage for specific positon, close cookies settings, load all of pages

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service)

def position_search(nazev_pozice:str): #name of position is automaticaly added on base of following codes
    url = f'https://nofluffjobs.com/pl/jobs/?criteria=keyword%3D"{nazev_pozice}"'
    browser.get(url)
    cookies_potvrzeny = False
    time.sleep(3)
    
    
    if not cookies_potvrzeny:
        cookie = browser.find_elements("id", "onetrust-accept-btn-handler")
        if len(cookie) > 0:
            print("Cookies byly nalezeny")
            cookie[0].click()   
            cookies_potvrzeny = True
        else:
            print("Cookies nebyly nalezeny")
        time.sleep(3)

    
    page_nr = 1
    
    
    try:
        next_page = browser.find_element("css selector", "tw-btn tw-btn-primary tw-px-8 tw-block tw-btn-xl")
        next_page.click()
        page_nr += 1
        time.sleep(5)
    except ElementClickInterceptedException:
        print("Tlačítko nebylo možné stisknout.")
        time.sleep(5)
    except NoSuchElementException:
        print("Tlačítko se již nezobrazuje, jsi na konci.")
                        
        
    print(f"Vsechny stranky byly nacteny. Celkem bylo nacteno {page_nr}")
        
    html = browser.page_source        
    return bs(html, "html.parser")


        

In [ ]:
#download positon name and web data

def uloz_data(nazev_pozice:str):
    result =  []
    web_data = position_search(nazev_pozice)
    result.append(
        {"nazev_pozice" : nazev_pozice,
         "stazena_data" : str(web_data)
         }
    )
    return result

In [ ]:
positions = ["data analyst", "data engineer", "data scientist"]

In [ ]:
data = []
for position in positions:
    print(data)
    data += uloz_data(position)
    print(f"Data pro pozici {position} byla uspesne ulozena.")

browser.close()

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
#getting HTML file with data from web for positions

i = 0
for zaznam in data:
    jmeno_souboru = zaznam["nazev_pozice"] #zde není název pozice proměnná, ale klíč ve slovníku (musí se to tedy shodovat s názvem klíče)

    with open(f"{jmeno_souboru}.html", "w", encoding = "utf-8") as file:
        file.write(zaznam["stazena_data"])
        print(f"Soubor pro pozici {jmeno_souboru} byl uspesne vytvoren.")